# Earthquake prediction using Random Forests
*Anders Poirel - 08/05/2019*


## Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import tsfresh as tsf
import glob 
import gc

**To-do:** show code from notebook with EDA.

## Feature Extraction

Functions for extraction predictors, response variables and test predictors from a **sample** of the dataset (by default, we sample every 100th entry in the `train.csv`)

In [ ]:
def get_predictors(filepath, col_name, seg_length, data_len, skip_amount = 100):
    series_df = pd.read_csv(filepath, 
                            usecols = [col_name],
                            dtype = np.float32
                            skiprows = range(1, data_len, skip_amount)
                           )
    num_points = len(series_df.index)
    interval_length = num_points // (seg_length // skip_amount)
    num_segs = num_points // interval_length 

    id_col = np.empty((num_points ,1))
    for i in range(num_segs):
        id_col[i] = i // interval_length

    series_df['id'] = id_col

    from tsfresh.feature_extraction import MinimalFCParameters

    predictors = tsf.extract_features(series_df,
                                      column_value = col_name,
                                      column_id = 'id',
                                     )
    del series_df
    gc.collect()

    return predictors.values[:, 1:]

In [ ]:
def get_responses(filepath, col_name, seg_len, data_len, skip_amount = 100):

    series_df = pd.read_csv(filepath, 
                            usecols = [col_name],
                            dtype = np.float32,
                            skiprows = range(1, data_len, skip_amount)
                           )
    num_points = len(series_df.index)
    interval_len = num_points // (seg_len // skip_amount)
    num_segs = num_points // interval_len

    response = np.empty(num_segs)
    for i in range(num_segs):
        response[i] = series_df.iloc[interval_len * (i+1), 0]
    del series_df
    gc.collect()

    return response

In [ ]:
def get_test_predictors(file_directory, col_name, seg_length, 
                        skip_amount = 100):

    test_predictors = []
    
    for fname in glob.glob(file_directory):
        seg_df = pd.read_csv(fname, skiprows = range(1,seg_length, skip_amount))
        id_col = np.zeroes(len(seg_df.index))
        seg_df['id'] = id_col
        temp_predictors = tsf.extract_features(seg_df,
                                               column_value = col_name,
                                               column_id = 'id')
        test_predictors.append(temp_predictors.iloc[1:])

    del seg_df
    gc.collect()

    return np.array(test_predictors)

Length of the training and test data files. We use these valus to simplify data extraction.

In [ ]:
DATA_LEN = 621985673 # counted using  a test run
SEG_LEN = 150001

Extract the data from `train.csv`

In [ ]:
y_train = get_responses('../input/train.csv',
                        col_name = 'time_to_failure',
                        seg_len = SEG_LEN,
                        data_len = DATA_LEN)


X_train = get_predictors('../input/train.csv',
                         col_name = 'acoustic_data',
                         seg_len = SEG_LEN,
                         data_len = DATA_LEN)

Save the results to a `.csv` file to simplify future modeling work.

In [ ]:
numpy.savetxt("y_train.csv", y_train, delimiter = ",")
numpy.savetxt("X_train.csv", X_train, delimiter = ",")

## Model Fit and Testing
Parameters for fitting decision trees. I used the recommendations in ISL as a guide to selecting these:

In [ ]:
MAX_DEPTH = 5
MAX_FEATURES = 'sqrt'

Fit the model:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth = MAX_DEPTH, max_features = MAX_FEATURES)
rf.fit(X_train, y_train)

### Model diagnostics
Compute accuracy of model predictions on training data

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_train)
accuracy_score(y_train, y_pred)

### Building the submission 

Extract test data and use model to make predictions:

In [ ]:
X_test_train = get_test_predictors(col_name = 'acoustic_data')
y_test_pred = rf.predict(X_test_train)

seg_names = []
for fname in glob.glob():
    seg_names.append(fname)

submission = pd.DataFrame('seg_id': seg_names, 'time_to_failure': y_test_pred)
submission.to_csv('submission.csv', index = False)

## Possible improvements
* using the entire dataset instead of a sample using methods for reducing the size of the data
* using scikit-learn's built-in parameter selection, which would require re-writing the entire data pipeline. Effect of any tuning using k-fold cross-validation will likely be minor as we are using random forests.
* building a strategy that leverages more the structure of the data - the data shows 17 earthquakes, and looks pseudo-periodical. I don't yet know how we can do this